# Import Libraries

In [1]:
import os
import pandas as pd
import torch
import numpy as np
from Preprocess import Process
from NeuralNetworkDIY import ClassifierNetwork, Model
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from PIL import Image

In [2]:
print(torch.cuda.is_available())

True


# Parameters for the dataset

In [3]:
RAW_DIR = "data/raw/140k faces"
PROCESSED_DIR = "data/processed"
IMAGE_SIZE=(128,128)

# Image Preprocessing

In [4]:
process=Process(RAW_DIR, PROCESSED_DIR, IMAGE_SIZE, delete_process_dir=True, batch_size=1000, aug_amount=0, filters=False)
process.run(0.95, 0.05)

Working on fake
{8000}


Processing valid: 100%|██████████| 8/8 [01:09<00:00,  8.70s/it]


Working on real
{3500}


Processing valid: 100%|██████████| 4/4 [00:32<00:00,  8.22s/it]


# DataLoader and Custom Dataset

In [9]:
class CustomImageDataset(Dataset):
    def __init__(self, directory):
        self.directory = directory
        self.image_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")

        transform = transforms.ToTensor()
        
        img= transform(img)
        # Extract label from the filename (fake -> 1, real -> 0)
        label = 1 if "fake" in os.path.basename(img_path).lower() else 0

        return img, label, img_path

In [10]:
train_data = CustomImageDataset(directory=f'{PROCESSED_DIR}/{IMAGE_SIZE[0]}/train/rgb')

train_loader = DataLoader(train_data, batch_size=1000, shuffle=True)

# Creating the network, model, training, predicting

In [11]:
network=ClassifierNetwork(IMAGE_SIZE)

In [12]:
model=Model(network, 20)

In [13]:
model.train(train_loader)


KeyboardInterrupt



In [ ]:
torch.save(model.state_dict(), 'ADL_model.pth')